In [1]:
import pandas as pd
import datetime

In [2]:
pop_by_zip=pd.read_csv("/home/jian/Docs/EASI/2018-07-19/ZIP4_18_DATA_A2_CSV/ZIP4_18_DATA_A2.CSV",dtype=str,skiprows=1,usecols=['ZIP_CODE','POP18','HH18'])
pop_by_zip['POP18']=pop_by_zip['POP18'].astype(float)
pop_by_zip['HH18']=pop_by_zip['HH18'].astype(float)
pop_by_zip=pop_by_zip.groupby(['ZIP_CODE'])['POP18','HH18'].sum().reset_index()
pop_by_zip=pop_by_zip.rename(columns={"ZIP_CODE":"zip_cd"})

In [3]:
IRI_Counties_both=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='Sheet1',dtype=str)
IRI_Counties_IRI=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='IRI',dtype=str,usecols=[0,1])
IRI_Counties_Nielsen=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='Nielsen',dtype=str,usecols=[0,1])
IRI_Counties_Nielsen.columns=[['County','cleaned_name']]
IRI_Counties_both=IRI_Counties_both[(IRI_Counties_both['IRI_Label']!='nan') & (IRI_Counties_both['Nielsen_Label']!='nan')]

IRI_Counties_both=IRI_Counties_both[['County','County']]
IRI_Counties_both.columns=['County','cleaned_name']

Nielsen_counties=IRI_Counties_both.append(IRI_Counties_Nielsen).drop_duplicates().reset_index()
IRI_counties=IRI_Counties_both.append(IRI_Counties_IRI).drop_duplicates().reset_index()

del Nielsen_counties['index']
del IRI_counties['index']


In [4]:
All_DMA_zips=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
All_DMA_zips=All_DMA_zips.iloc[:,[0,2,6,9]]
All_DMA_zips.columns=['zip_cd','DMA','State','County']
All_DMA_zips['County']=All_DMA_zips['County']+"|"+All_DMA_zips['State']
All_DMA_zips['County']=All_DMA_zips['County'].apply(lambda x: x.replace(" ","").lower())
All_DMA_zips=All_DMA_zips[~All_DMA_zips['State'].isin(['AK','HI'])]

In [5]:
All_IRI_counties=pd.read_excel("/home/jian/SunnyD/Otherinput/Total_US_counties_IRI_markets_and_regions_cr_12272016.xlsx",dtype=str,sheetname='U.S. Counties',skiprows=1)
All_IRI_counties=All_IRI_counties[['County Name','State','IRI Market']]
All_IRI_counties['County']=All_IRI_counties['County Name']+"|"+All_IRI_counties['State']
All_IRI_counties['County']=All_IRI_counties['County'].apply(lambda x: x.replace(" ","").lower())
All_IRI_counties=pd.merge(All_IRI_counties,IRI_counties,on="County",how="left")
All_IRI_counties=All_IRI_counties[['IRI Market','cleaned_name']]

In [6]:
All_DMA_zips=pd.merge(All_DMA_zips,Nielsen_counties,on="County",how="left")
All_DMA_zips=All_DMA_zips[['zip_cd','DMA','County','cleaned_name']].rename(columns={"County":"Nielsen_County"})

In [7]:
All_DMA_zips_merge=pd.merge(All_DMA_zips,All_IRI_counties,on="cleaned_name",how="left")
All_DMA_zips_merge=pd.merge(All_DMA_zips_merge,pop_by_zip,on="zip_cd",how="left")

In [8]:
output=pd.DataFrame()
for DMA,group in All_DMA_zips_merge.groupby(['DMA']):
    DMA_total_Pop=group[['zip_cd','POP18']].drop_duplicates()['POP18'].sum()
    
    DMA_by_Zip_IRI=group[['zip_cd','IRI Market']].drop_duplicates().groupby(['zip_cd'])['IRI Market'].count().to_frame().reset_index().rename(columns={"IRI Market":"IRI_count"})
    zip_DMA=group[['zip_cd','DMA']].drop_duplicates()
    
    df=group[['zip_cd','DMA','IRI Market','POP18']].drop_duplicates()
    df=pd.merge(df,DMA_by_Zip_IRI,on='zip_cd',how='left')
    df['POP18_weighted']=df['POP18']/df['IRI_count']
    if abs(df['POP18_weighted'].sum()-DMA_total_Pop)>0.1:
        print(DMA,df['POP18_weighted'].sum(),DMA_total_Pop)
    allocated_DMA=df.groupby(['DMA','IRI Market'])['POP18_weighted'].sum().to_frame().reset_index()
    allocated_DMA['Pctg']=allocated_DMA['POP18_weighted']/DMA_total_Pop
    output=output.append(allocated_DMA)

In [9]:
output.to_csv("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/from_DMA_to_IRI_based_on_common_population.csv")

In [25]:
media_spend=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/FB-IG_Sunny D_Spend by DMA & Day_8.21.2018.xlsx",sheetname="Raw Data Report",dtype=str,skiprows=2)
media_spend=media_spend[['DMA Region','Day','Impressions','Amount Spent']]
media_spend=media_spend[~media_spend['Day'].isin(['nan','All'])]
media_spend['Amount Spent']=media_spend['Amount Spent'].astype(float)
media_spend['Impressions']=media_spend['Impressions'].astype(int)
media_spend=media_spend.groupby(['DMA Region','Day'])['Impressions','Amount Spent'].sum().reset_index()


66325.390000000043

In [26]:
media_spend_NAME=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/FB-IG_Sunny D_Spend by DMA & Day_8.21.2018.xlsx",sheetname="DMA_Name_Match",dtype=str)
media_spend_NAME.columns=['DMA Region','DMA']

In [27]:
media_spend=pd.merge(media_spend,media_spend_NAME,on="DMA Region",how="left")

In [28]:
media_spend_IRI=pd.merge(media_spend,output,on="DMA",how="left")
media_spend_IRI['Cost_allocated']=media_spend_IRI['Amount Spent']*media_spend_IRI['Pctg']
media_spend_IRI['Impr_allocated']=media_spend_IRI['Impressions']*media_spend_IRI['Pctg']

In [29]:
media_spend_IRI=media_spend_IRI.groupby(['DMA Region','DMA','Day','IRI Market'])['Cost_allocated','Impr_allocated'].sum().reset_index()

In [49]:
not_allocated=media_spend[media_spend['DMA Region']=="Non-DMA Region"]

In [46]:
writer=pd.ExcelWriter("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/allocated_cost.xlsx",engine="xlsxwriter")

In [50]:
media_spend_IRI.to_excel(writer,"allocated_cost_withinDMA",index=False)
not_allocated.to_excel(writer,"Non_DMA_Region",index=False)
output.to_excel(writer,"proportion_on_pop",index=False)
writer.save()